In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import shutil

# from __future__ import print_function

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import sklearn.model_selection
from sklearn.metrics import accuracy_score, mean_squared_error 
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data.dataloader import default_collate
from pytorchtools import EarlyStopping

In [2]:
class MyDataset(Dataset):
    
    def __init__(self,x,y,training_type):

        self.x_train=torch.tensor(x,dtype=torch.float32)
        if training_type == 'classify':
            self.y_train=torch.tensor(y,dtype=torch.int64)
        elif training_type == 'regress':
            self.y_train=torch.tensor(y,dtype=torch.float32)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [3]:
class NetCNN(nn.Module):
    def __init__(self,dim_num,classes):
        super(NetCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(dim_num, 64, kernel_size=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=0.3)
        )
        self.layer1_1 = nn.Sequential(
            nn.Conv1d(64, 16, kernel_size=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=0.3)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 8, kernel_size=3),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Dropout(p=0.3)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(8, 48),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )
        
        self.layer3_1 = nn.Sequential(
            nn.Linear(48, 20),
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )
        
        self.layer4 = nn.Linear(in_features=20, out_features=classes)
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer1_1(out)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer3(out)
        out = self.layer3_1(out)
        out = self.layer4(out)

        
        return out

In [4]:
class NetLSTM(nn.Module):
    def __init__(self,dim_num,classes):
        super(NetLSTM, self).__init__()
        
        self.layer1 = nn.LSTM(input_size=dim_num,
                            hidden_size=16,
                            num_layers=3,
                            batch_first=True,
                            bidirectional=True)
        
        self.layer4 = nn.Linear(in_features=640, out_features=classes)
        

    def forward(self, x):
        out, states = self.layer1(x)
        out = out.reshape(out.size(0), -1)   # Flatten them for Many to one LSTM
        out = self.layer4(out)
        
        return out

In [5]:
class NetTransf(nn.Module):
    def __init__(self, dim_num: int, d_model: int, nhead: int, d_hid: int, 
                 nlayers: int, classes: int, dropout: float, batch_size: int):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
#         self.encoder = nn.Linear(ntoken, d_model)
        self.d_model = d_model
#         self.dim_num = dim_num
        self.decoder = nn.Linear(dim_num, classes)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
#         self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = src * math.sqrt(self.d_model)
#         print(src.shape)
        src = src.permute(2, 0, 1)
#         print(src.shape)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
#         print(output.shape)
        output = output.mean(0)
#         print(output.shape)
        output = self.decoder(output)
        return output
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
    
# def generate_square_subsequent_mask(sz: int) -> Tensor:
#     """Generates an upper-triangular matrix of -inf, with zeros on diag."""
#     return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [6]:
def main_to_run(data_train, data_test, input_columns, mode, training_type, number_epochs, batch_size, learning_rate, fine_tuning,fine_tune_ckp_path, inference_only, inference_ckp_path, regress_type):

    X_train = data_train[input_columns]
    X_test = data_test[input_columns]
    
    if training_type == 'classify':
        y_train = data_train.POIBuilding_new
        y_test = data_test.POIBuilding_new
        output_classes = 16
        
    elif training_type == 'regress':
        y_train = data_train[['PoI_GT_centerLatAngle',
                      'PoI_GT_visibleMinLatAngle','PoI_GT_visibleMaxLatAngle','PoI_GT_minLatAngle',
                      'PoI_GT_maxLatAngle','PoI_GT_adjustedMinLatAngle','PoI_GT_adjustedMaxLatAngle']]
        y_test = data_test[['PoI_GT_centerLatAngle',
                    'PoI_GT_visibleMinLatAngle','PoI_GT_visibleMaxLatAngle','PoI_GT_minLatAngle',
                    'PoI_GT_maxLatAngle','PoI_GT_adjustedMinLatAngle','PoI_GT_adjustedMaxLatAngle']]
        output_classes = 1

    if mode == 'CNN' or mode == 'Transformer':
        X_train = (X_train.values.reshape(X_train.shape[0],X_train.shape[-1]//20,20))#.swapaxes(1,2)
        X_test = (X_test.values.reshape(X_test.shape[0],X_test.shape[-1]//20,20))#.swapaxes(1,2)
    
    elif mode == 'LSTM':
        X_train = (X_train.values.reshape(X_train.shape[0],X_train.shape[-1]//20,20)).swapaxes(1,2)
        X_test = (X_test.values.reshape(X_test.shape[0],X_test.shape[-1]//20,20)).swapaxes(1,2)
    
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
       X_train, y_train, random_state=42, test_size=0.12
    )
    
    print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
    print("#"*80)
    
    if mode == 'Transformer':
        ntokens = X_train.shape[1]  # size of vocabulary
        emsize = 8  # embedding dimension
        d_hid = 50  # dimension of the feedforward network model in nn.TransformerEncoder
        nlayers = 6  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
        nhead = 8  # number of heads in nn.MultiheadAttention
        dropout = 0.2  # dropout probability
        net = NetTransf(ntokens, emsize, nhead, d_hid, nlayers, output_classes, dropout, batch_size)

    if mode == 'CNN':
        net = NetCNN(X_train.shape[1],classes=output_classes)
    
    elif mode == 'LSTM':
        net = NetLSTM(X_train.shape[-1],classes=output_classes)
    
    print(net)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Assuming that we are on a CUDA machine, this should print a CUDA device:
    print(device)
    # In[17]:
    net.to(device)


    def save_ckp(state, is_best,f_path):
        torch.save(state, f_path)
        if is_best:
            best_fpath = 'best_model.pt'
            shutil.copyfile(f_path, best_fpath)

    def load_ckp(checkpoint_fpath, model, optimizer):
        checkpoint = torch.load(checkpoint_fpath)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer, checkpoint['epoch']
    
    #padding nan values in X
#     X_train = np.nan_to_num(X_train)
#     X_test = np.nan_to_num(X_test)
#     X_val = np.nan_to_num(X_val)
    
    
    if training_type == 'classify':
        y_train = y_train.values.reshape(-1,1)
        y_val = torch.squeeze(torch.tensor(y_val.values.reshape(-1,1),dtype=torch.int64)).to(device)
        dataset = MyDataset(X_train, y_train,training_type)
        
    elif training_type == 'regress' and regress_type == 'regress_center':
        print('regress_center')
        y_train_center_only = y_train.PoI_GT_centerLatAngle.values.reshape(-1,1)
        y_val_center_only = torch.tensor(y_val.PoI_GT_centerLatAngle.values.reshape(-1,1),dtype=torch.float32).to(device)
        dataset = MyDataset(X_train, y_train_center_only,training_type)
    
    elif regress_type == 'regress_facade':
        print('regress_facade')
        y_train_center_only = (y_train[['PoI_GT_visibleMinLatAngle','PoI_GT_visibleMaxLatAngle']].mean(axis=1)).values.reshape(-1,1)
        y_val_center_only = torch.tensor((y_val[['PoI_GT_visibleMinLatAngle','PoI_GT_visibleMaxLatAngle']].mean(axis=1)).values.reshape(-1,1),dtype=torch.float32).to(device)
        dataset = MyDataset(X_train, y_train_center_only,training_type)

    #create dataset and generate dataloader
    
    data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, 
                             collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

    # Define Optimizer and Loss Function
#     optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate)
    

    
    if training_type == 'classify':
        loss_func = torch.nn.CrossEntropyLoss()
    elif training_type == 'regress':
        loss_func = torch.nn.MSELoss()
    
    #saving checkpoints for continuing training (e.g., transfer learning)
    
    if not inference_only:
    
        if fine_tuning:
            net, optimizer, start_epoch = load_ckp(fine_tune_ckp_path, net, optimizer)
            print(start_epoch)

        print('Training the Deep Learning network ...')



        val_losses = []
        train_losses = []


        # initialize the early_stopping object
        early_stopping = EarlyStopping(patience=40, verbose=True)



        for epoch in range(number_epochs):
            print(epoch)
            avg_cost = []

            net.train()
            for i, (batch_X, batch_Y) in enumerate(data_loader):
                X = Variable(batch_X)    
                Y = Variable(batch_Y)
                if training_type == 'classify':
                    Y = torch.squeeze(Y)

                optimizer.zero_grad() # <= initialization of the gradients

                # forward propagation
                prediction = net(X)
    #             print(prediction.shape, Y.shape)
                loss = loss_func(prediction, Y) # <= compute the loss function

                # Backward propagation
                loss.backward() # <= compute the gradient of the loss/cost function     
                optimizer.step() # <= Update the gradients


            checkpoint = {
                'epoch': epoch + 1,
                'state_dict': net.state_dict(),
                'optimizer': optimizer.state_dict()
            }

            if fine_tuning:
                save_ckp(checkpoint, False, 'checkpoint_finetuned.pt')
            else:
                save_ckp(checkpoint, False, 'checkpoint.pt')

            train_losses.append(loss.item())
    #         print('train loss:', loss.item())

            net.eval() # prep model for evaluation
            with torch.no_grad():
    #             print(X_val)
                X_val_trch = torch.tensor(X_val,dtype=torch.float32).to(device)
    #             print(X_val_trch.shape)
                prediction_val = net(X_val_trch)
    #             print(prediction_val,y_val_center_only)
                if training_type == 'classify':
                    loss_val = loss_func(prediction_val, y_val)
                elif training_type == 'regress':
                    loss_val = loss_func(prediction_val, y_val_center_only)
                val_losses.append(loss_val.item())
    #             print('val loss:', loss_val.item())

            # early_stopping needs the validation loss to check if it has decresed, 
            # and if it has, it will make a checkpoint of the current model
            early_stopping(np.average(val_losses), net)
            print(np.average(val_losses))
            if early_stopping.early_stop:
                save_ckp(checkpoint, True, 'checkpoint_earlystopping_full.pt')
                print("Early stopping at Epoch " + str(epoch))

        print('Learning Finished!')

        print('last train loss:', loss.item())
        print('last val loss:', loss_val.item())

        plt.figure(figsize=(10,5))
        plt.title("Training and Validation Loss")
        plt.plot(val_losses,label="val")
        plt.plot(train_losses,label="train")
        plt.xlabel("iterations")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        
#     if inference_only:
#         net.load_state_dict(torch.load(inference_ckp_path))
#         net.eval()
        
    if inference_only:
        
        if "earlystop" in inference_ckp_path:
            net.load_state_dict(torch.load(inference_ckp_path))
        else:
            net, optimizer, start_epoch = load_ckp(inference_ckp_path, net, optimizer)
            print(start_epoch)
            
        net.eval()
    
    X_train_trch = torch.from_numpy(X_train).float().to(device)
    np.savetxt('X_train.csv',X_train.reshape(X_train.shape[0], -1), delimiter=",")
    print(X_train_trch.shape)
    with torch.no_grad():
        prediction_train = net(X_train_trch).cpu()
    
    if training_type == 'classify':
        softmax = torch.exp(prediction_train).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)

        # accuracy on training set
        final_accuracy = accuracy_score(y_train, predictions)
        print('Train Accuracy: ', final_accuracy)

    elif training_type == 'regress':
        print(prediction_train.shape)
        prediction_np = prediction_train.detach().numpy().reshape(len(prediction_train))
        
        y_train['PoI_GT_centerLatAngle_predicted'] = prediction_np
        
        #generate list of indices for incremental learning, use K when loading to use
        print("(((((((())))))))")
        print(y_train.columns)
        y_train['prediction_error'] = (y_train['PoI_GT_centerLatAngle'] - y_train['PoI_GT_centerLatAngle_predicted']).abs()
        temp_y_train = y_train.sort_values(by='prediction_error').reset_index(names='priority_index')

        # uncomment the following if you want to save a new priotirized examplers and move the file to the Data folder
#         temp_y_train['priority_index'].to_csv('y_train_indeces_priotirized.csv')
        print("(((((((())))))))")
        
        final_accuracy = len(y_train[(y_train.PoI_GT_centerLatAngle_predicted > y_train.PoI_GT_minLatAngle) 
               & (y_train.PoI_GT_centerLatAngle_predicted < y_train.PoI_GT_maxLatAngle)]) / len(y_train)
        print('Train Accuracy MRDE: ', final_accuracy)

        final_accuracy = len(y_train[(y_train.PoI_GT_centerLatAngle_predicted > y_train.PoI_GT_visibleMinLatAngle) 
               & (y_train.PoI_GT_centerLatAngle_predicted < y_train.PoI_GT_visibleMaxLatAngle)]) / len(y_train)
        print('Train Accuracy SegObj: ', final_accuracy)

        final_accuracy = len(y_train[(y_train.PoI_GT_centerLatAngle_predicted > y_train.PoI_GT_adjustedMinLatAngle) 
               & (y_train.PoI_GT_centerLatAngle_predicted < y_train.PoI_GT_adjustedMaxLatAngle)]) / len(y_train)
        print('Train Accuracy MinDT: ', final_accuracy)

    X_val_trch = torch.from_numpy(X_val).float().to(device)
    print(X_val_trch.shape)
    with torch.no_grad():
        prediction_val = net(X_val_trch).cpu()
    
    if training_type == 'classify':
        softmax = torch.exp(prediction_val).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)

        # accuracy on training set
        final_accuracy = accuracy_score(y_val, predictions)
        print('Val Accuracy: ', final_accuracy)

    elif training_type == 'regress':
        print(prediction_val.shape)
        prediction_np = prediction_val.detach().numpy().reshape(len(prediction_val))

        y_val['PoI_GT_centerLatAngle_predicted'] = prediction_np
        final_accuracy = len(y_val[(y_val.PoI_GT_centerLatAngle_predicted > y_val.PoI_GT_minLatAngle) 
               & (y_val.PoI_GT_centerLatAngle_predicted < y_val.PoI_GT_maxLatAngle)]) / len(y_val)
        print('Val Accuracy MRDE: ', final_accuracy)

        final_accuracy = len(y_val[(y_val.PoI_GT_centerLatAngle_predicted > y_val.PoI_GT_visibleMinLatAngle) 
               & (y_val.PoI_GT_centerLatAngle_predicted < y_val.PoI_GT_visibleMaxLatAngle)]) / len(y_val)
        print('Val Accuracy SegObj: ', final_accuracy)

        final_accuracy = len(y_val[(y_val.PoI_GT_centerLatAngle_predicted > y_val.PoI_GT_adjustedMinLatAngle) 
               & (y_val.PoI_GT_centerLatAngle_predicted < y_val.PoI_GT_adjustedMaxLatAngle)]) / len(y_val)
        print('Val Accuracy MinDT: ', final_accuracy)


    X_test_trch = torch.from_numpy(X_test).float().to(device)
    print(X_test_trch.shape)
    with torch.no_grad():
        prediction_test = net(X_test_trch).cpu()
        
        
    if training_type == 'classify':
        softmax = torch.exp(prediction_test).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)

        # accuracy on test set
        final_accuracy_test = accuracy_score(y_test, predictions)
        print('Test Accuracy: ', final_accuracy_test)
        
        return final_accuracy_test

    elif training_type == 'regress':
        print(prediction_test.shape)
        prediction_test_np = prediction_test.detach().numpy().reshape(len(prediction_test))
        
        y_test['PoI_GT_centerLatAngle_predicted'] = prediction_test_np
        
        performance_askl = sklearn.metrics.mean_squared_error(y_test[['PoI_GT_visibleMinLatAngle','PoI_GT_visibleMaxLatAngle']].mean(axis=1), y_test['PoI_GT_centerLatAngle_predicted'], squared=False)
        print(f"CNN Regressor MSE is {performance_askl}")
        print(math.degrees(performance_askl))
    
        final_accuracy_latangle = len(y_test[(y_test.PoI_GT_centerLatAngle_predicted > y_test.PoI_GT_minLatAngle) 
               & (y_test.PoI_GT_centerLatAngle_predicted < y_test.PoI_GT_maxLatAngle)]) / len(y_test)
        print('Test Accuracy MRDE: ', final_accuracy_latangle)

        final_accuracy_visiblelatangle = len(y_test[(y_test.PoI_GT_centerLatAngle_predicted > y_test.PoI_GT_visibleMinLatAngle) 
               & (y_test.PoI_GT_centerLatAngle_predicted < y_test.PoI_GT_visibleMaxLatAngle)]) / len(y_test)
        print('Test Accuracy SegObj: ', final_accuracy_visiblelatangle)

        final_accuracy_adjustedlatangle = len(y_test[(y_test.PoI_GT_centerLatAngle_predicted > y_test.PoI_GT_adjustedMinLatAngle) 
               & (y_test.PoI_GT_centerLatAngle_predicted < y_test.PoI_GT_adjustedMaxLatAngle)]) / len(y_test)
        print('Test Accuracy MinDT: ', final_accuracy_adjustedlatangle)

        return final_accuracy_latangle, final_accuracy_visiblelatangle, final_accuracy_adjustedlatangle

    


    

In [7]:
folder_path = "../Data/"
mode = 'CNN' #CNN or LSTM or Transformer network
training_type = 'regress' #classify or regress
regress_type = 'regress_center' #regress_facade or regress_center
batch_size = 128
number_epochs = 20000
learning_rate = 0.0001
fine_tune_or_not = False
fine_tune_ckp_path = "checkpoint.pt"
inference_only = True
# inference_ckp_path = "checkpoint.pt"
inference_ckp_path = "checkpoint_earlystop.pt"
exemplar_ratio = 0 # 2 is half, 4 is quarter and so on while 0 is naive finetuning

In [8]:
def multiple_runs_concatenate(file_path1, file_path2 = False, exemplars_list_path = False):
    
    data = pd.read_csv(folder_path + file_path1).iloc[:,2:]
    data['POIBuilding_new'] = data.POIBuilding_new - 1
#     print(data)
    
    
    
    #dropping test for the currently loaded variant and loading it from data2
    data = data[~((data.Participant == 'P45') 
                      | (data.Participant == 'P75') 
                      | (data.Participant == 'P22') 
                      | (data.Participant == 'P40')
                      | (data.Participant == 'P57')
                      | (data.Participant == 'P74')
                       )]
    
    if file_path2 != False:
        print('A second File Path was given for the test Data.')
        print(file_path2)
        data2 = pd.read_csv(folder_path + file_path2).iloc[:,2:]
        data2['POIBuilding_new'] = data2.POIBuilding_new - 1
        
        #left is 22,45,74
        #right is 40,57,75
        #lessthan4 is 40,74,75
        #morethan6 is 22,45,57
        
        test_data_index_in_IsNoise = data2[(
                        (data2.Participant == 'P45') 
                      | (data2.Participant == 'P75') 
                      | (data2.Participant == 'P22') 
                      | (data2.Participant == 'P40')
                      | (data2.Participant == 'P57')
                      | (data2.Participant == 'P74')
                     )].index.tolist()
        
        
        if exemplars_list_path != False:
            exemplars_temp = pd.read_csv(folder_path + exemplars_list_path).iloc[:,1:]
            
            if exemplar_ratio != 0:
                exemplar_number = len(exemplars_temp) // exemplar_ratio
                exemplars_list = exemplars_temp['priority_index'].iloc[:exemplar_number].tolist()
                print(len(exemplars_list))
#                 exemplars_list2 = exemplars_temp['priority_index'].iloc[-exemplar_number:].tolist()
#                 exemplars_list.extend(exemplars_list2)
                exemplars_list.extend(test_data_index_in_IsNoise)
                print(len(exemplars_list))
            elif exemplar_ratio == 0:
                exemplars_list = test_data_index_in_IsNoise
                print(len(exemplars_list))
                
            print(exemplars_list)
            data2 = data2.iloc[exemplars_list]
            
        
        data = pd.concat([data,data2])
        
        
    data.drop_duplicates(inplace=True)
    data.reset_index(inplace=True)
#     print(data)
    #     print(data.columns)
    print("#"*80)

    # ffill and bfill instead of replacing np.nan with zeros
    data[[col for col in data.columns if 'PointingDirectionX_' in col]] = data[[col for col in data.columns if 'PointingDirectionX_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'PointingDirectionZ_' in col]] = data[[col for col in data.columns if 'PointingDirectionZ_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'HeadGazeDirectionX_' in col]] = data[[col for col in data.columns if 'HeadGazeDirectionX_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'HeadGazeDirectionZ_' in col]] = data[[col for col in data.columns if 'HeadGazeDirectionZ_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'HeadposeX_' in col]] = data[[col for col in data.columns if 'HeadposeX_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'HeadposeZ_' in col]] = data[[col for col in data.columns if 'HeadposeZ_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'GazeX_' in col]] = data[[col for col in data.columns if 'GazeX_' in col]].ffill(axis=1).bfill(axis=1)
    data[[col for col in data.columns if 'GazeZ_' in col]] = data[[col for col in data.columns if 'GazeZ_' in col]].ffill(axis=1).bfill(axis=1)
    data.dropna(inplace=True)
    

    data_train = data[~((data.Participant == 'P45') 
                      | (data.Participant == 'P75') 
                      | (data.Participant == 'P22') 
                      | (data.Participant == 'P40')
                      | (data.Participant == 'P57')
                      | (data.Participant == 'P74')
                       )]
    data_test = data[((data.Participant == 'P45') 
                      | (data.Participant == 'P75') 
                      | (data.Participant == 'P22') 
                      | (data.Participant == 'P40')
                      | (data.Participant == 'P57')
                      | (data.Participant == 'P74')
                     )]
    
    input_columns = []
    input_columns.extend([col for col in data_test.columns if ('PointingDirectionX_' in col) or ('PointingDirectionZ_' in col)])
    input_columns.extend([col for col in data_test.columns if ('HeadGazeDirectionX_' in col) or ('HeadGazeDirectionZ_' in col)])
    input_columns.extend([col for col in data_test.columns if ('HeadposeX_' in col) or ('HeadposeZ_' in col)])
    input_columns.extend([col for col in data_test.columns if ('GazeX_' in col) or ('GazeZ_' in col)])

#     input_columns = [col for col in data_test.columns if ('X_' in col) or ('Z_' in col)]

#     print(len(input_columns))
#     print(data_train.columns)
    
    
    if training_type == 'classify':
    
        all_runs_acc = []


        for i in range(1):
            
            final_accuracy = main_to_run(data_train,data_test,input_columns,mode,training_type,number_epochs,batch_size,learning_rate,fine_tune_or_not,fine_tune_ckp_path,inference_only, inference_ckp_path, regress_type)
            all_runs_acc.append(final_accuracy)

        print("#"*80)
        print("#"*80)
        print(all_runs_acc)
        print((np.asarray(all_runs_acc)).mean())
        print("#"*80)
        print("#"*80)
    
    elif training_type == 'regress':
#     else:
        
        all_runs_latangle = []
        all_runs_visiblelatangle = []
        all_runs_adjustedlatangle = []

        for i in range(1):
            
            final_accuracy_latangle, final_accuracy_visiblelatangle, final_accuracy_adjustedlatangle = main_to_run(data_train,data_test,input_columns,mode,training_type,number_epochs,batch_size,learning_rate,fine_tune_or_not,fine_tune_ckp_path,inference_only, inference_ckp_path, regress_type)
            
            all_runs_latangle.append(final_accuracy_latangle)
            all_runs_visiblelatangle.append(final_accuracy_visiblelatangle)
            all_runs_adjustedlatangle.append(final_accuracy_adjustedlatangle)

        print("#"*80)
        print("#"*80)
        print(all_runs_latangle, all_runs_visiblelatangle, all_runs_adjustedlatangle)
        print((np.asarray(all_runs_latangle)).mean(), (np.asarray(all_runs_visiblelatangle)).mean() ,(np.asarray(all_runs_adjustedlatangle)).mean())
        print("#"*80)
        print("#"*80)



In [9]:
multiple_runs_concatenate('Data_based_on_gesture_onset.csv','/Data_based_on_Speech_onset.csv')

A second File Path was given for the test Data.
/Data_based_on_Speech_onset.csv
################################################################################
(3414, 8, 20) (275, 8, 20) (3414, 7) (275, 7)
(3004, 8, 20) (410, 8, 20) (275, 8, 20) (3004, 7) (410, 7) (275, 7)
################################################################################
NetCNN(
  (layer1): Sequential(
    (0): Conv1d(8, 64, kernel_size=(2,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (layer1_1): Sequential(
    (0): Conv1d(64, 16, kernel_size=(2,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (layer2): S

/tmp/ipykernel_185071/4268575643.py:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test['PoI_GT_centerLatAngle_predicted'] = prediction_test_np
